In [10]:
import numpy as np
import cvxpy as cp

# Exchange rate data.
tickers = ["USD", "EUR", "GBP", "CAD", "JPY", "CNY", "RUB", "MXN", "INR", "BRL"]
n = 10 

F = np.zeros((n, n))

# USD
data = ([1.0, 0.87, 0.76, 1.31, 108.90, 6.72, 65.45, 19.11, 71.13, 3.69],
# EUR
[1.0, 0.88, 1.51, 125.15, 7.72, 75.23, 21.96, 81.85, 4.24],
# GBP
[1.0, 1.72, 142.94, 8.82, 85.90, 25.08, 93.50, 4.84],
# CAD
[1.0, 82.93, 5.11, 49.82, 14.54, 54.23, 2.81],
# JPY
[1.0, 0.062, 0.60, 0.18, 0.65, 0.034],
# CNY
[1.0, 9.74, 2.85, 10.61, 0.55],
# RUB
[1.0, 0.29, 1.09, 0.056],
# MXN
[1.0, 3.73, 0.19],
# INR
[1.0, 0.052],
# BRL
[1.0])
for i in range(n):
    F[i,i:] = data[i]
for j in range(n):
    for i in range(j+1,n):
        F[i,j] = 1.035/F[j,i]
        
# Initial and final portfolios.
c_req = np.arange(1,n+1)
c_req = 1e4*c_req/c_req.sum()
c_init = c_req[::-1]


In [11]:
X = cp.Variable((n,n))
z = (X/F)@np.ones(n) - X.T@np.ones(n) + c_init
cost = (c_init - z)@np.sqrt(F[:,0]/F[0,:])
problem = cp.Problem(cp.Minimize(cost),[X >= 0,cp.diag(X) == 0,X.T@np.ones(n) <= c_init,z >= c_req])
result = problem.solve()
print(" The minimum currency exchange cost is", result, " USD")

 The minimum currency exchange cost is 7.720059340057958  USD


In this given problem we have 10 currencies i.e. "USD, EUR, GBP, CAD,JPY,CNY,RUB,MXN,INR,BRL" and **c_int** is a vector that contain the number of units of respective currency our goal is to exchange currency such that after exchange there are at least **c_req** units of respective currency.

**X_ij**="Exchange currency j into i"
**z**="currency holdings after exchange"

These X and z are optimization variables.

**Constraints:**

                  1. X >= 0 

                  2. cp.diag(X) == 0

                  3. X.T@np.ones(n) <= c_init
                
                  4. z >= c_req